In [1]:
import pandas as pd #импортируем библиотеку pandas

In [2]:
# считываем датасет и сохраняем в переменную bookings
bookings = pd \
    .read_csv('/home/jupyter-n-demidov-20/MINIPROJECTS/2_bookings.csv', sep=';')

In [5]:
bookings_head = bookings.head(7) #записываем первые 7 строк в переменную

In [8]:
bookings.shape

(119390, 21)

In [133]:
bookings.dtypes #посмотрим, какой тип данных встречается чаще всего

hotel                         object
is_canceled                    int64
lead_time                      int64
arrival_full_date             object
arrival_date_year              int64
arrival_date_month            object
arrival_date_week_number       int64
arrival_date_day_of_month      int64
stays_in_weekend_nights        int64
stays_in_week_nights           int64
stays_total_nights             int64
adults                         int64
children                     float64
babies                         int64
meal                          object
country                       object
reserved_room_type            object
assigned_room_type            object
customer_type                 object
reservation_status            object
reservation_status_date       object
total_kids                   float64
has_kids                        bool
dtype: object

In [19]:
bookings.columns = bookings.columns.str.lower().str.replace(' ', '_')
# переименовываем колонки, удобный для работы, то есть делаем все буквы маленькими 
# и меняем пробелы на нижние подчеркивания

In [21]:
bookings.head()

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


In [ ]:
# попробуем узнать, пользователи из каких стран совершили наибольшее число успешных бронирований( найдем топ-5)

In [23]:
success_bookings = bookings.query('is_canceled == 0') #выводим успешные бронирования

In [26]:
# группируем по стране и считаем количество бронирований
success_bookings.groupby('country', as_index=False) \
    .agg({'is_canceled' : 'count'}) \
    .sort_values('is_canceled', ascending=False)

,country,is_canceled
125,PRT,21071
57,GBR,9676
54,FRA,8481
50,ESP,6391
42,DEU,6069
...,...,...
138,SMR,1
101,MLI,1
136,SLE,1
94,MAC,1


In [138]:
# второй вариант решения, более лаконичный(спасибо лучшему питонисту)

In [139]:
bookings.query('is_canceled == 0').country.value_counts()[:5]

PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: country, dtype: int64

In [29]:
bookings['stays_total_nights']

0         0
1         0
2         1
3         1
4         2
         ..
119385    7
119386    7
119387    7
119388    7
119389    9
Name: stays_total_nights, Length: 119390, dtype: int64

In [32]:
bookings.hotel.unique() #посмотрели, сколько всего типов отелей 

array(['Resort Hotel', 'City Hotel'], dtype=object)

In [43]:
# посмотрим, на сколько ночей в среднем бронируют отели типа Резорт и Сити
# также округлим среднее с помощью функции round
mean_stays_total_nights_by_hotel = bookings.groupby('hotel', as_index=False) \
                                           .agg({'stays_total_nights': 'mean'}) \
                                           .round(2)

In [44]:
mean_stays_total_nights_by_hotel

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


In [ ]:
# посмотрим ,сколько типов номера, присвоенных клиенту, отличается от изначально забронированного
# assigned_room_type от reserved_room_type

In [49]:
bookings[['assigned_room_type', 'reserved_room_type']].head(5)

,assigned_room_type,reserved_room_type
0,C,C
1,C,C
2,C,A
3,A,A
4,A,A


In [50]:
bookings.query('assigned_room_type != reserved_room_type').shape[0]

14917

In [51]:
#Теперь проанализируем даты запланированного прибытия (arrival_date_year). 
#На какой месяц чаще всего оформляли бронь в 2016 году? Изменился ли самый популярный месяц в 2017?

In [143]:
bookings.query('arrival_date_year == 2016 or arrival_date_year == 2017') \
        .groupby(['arrival_date_year', 'arrival_date_month']) \
        .agg({'is_canceled': 'count'}) \
        .sort_values(['arrival_date_year', 'is_canceled'], ascending=False)

is_canceled
arrival_date_year arrival_date_month             
2017              May                        6313
                  April                      5661
                  June                       5647
                  July                       5313
                  March                      4970
                  August                     4925
                  February                   4177
                  January                    3681
2016              October                    6203
                  May                        5478
                  April                      5428
                  September                  5394
                  June                       5292
                  August                     5063
                  March                      4824
                  July                       4572
                  November                   4454
                  February                   3891
                  December                   3860
                  January                    2248

In [70]:
# второй вариант решения через value_counts

In [73]:
bookings.query('arrival_date_year == 2016').arrival_date_month.value_counts()

October      6203
May          5478
April        5428
September    5394
June         5292
August       5063
March        4824
July         4572
November     4454
February     3891
December     3860
January      2248
Name: arrival_date_month, dtype: int64

In [74]:
bookings.query('arrival_date_year == 2017').arrival_date_month.value_counts()

May         6313
April       5661
June        5647
July        5313
March       4970
August      4925
February    4177
January     3681
Name: arrival_date_month, dtype: int64

In [ ]:
# Сгруппируем данные по годам, а затем проверим, на какой месяц (arrival_date_month) 
# бронирования отеля типа City Hotel отменялись чаще всего.

In [144]:
bookings.query('is_canceled == 1 and hotel == "City Hotel"') \
        .groupby(['arrival_date_year', 'arrival_date_month']) \
        .agg({'is_canceled': 'count'}) \
        .sort_values(['arrival_date_year', 'is_canceled'], ascending=False)

is_canceled
arrival_date_year arrival_date_month             
2017              May                        2217
                  April                      1926
                  June                       1808
                  July                       1324
                  March                      1278
                  August                     1123
                  January                    1044
                  February                    971
2016              October                    1947
                  June                       1720
                  September                  1567
                  April                      1539
                  May                        1436
                  November                   1360
                  August                     1247
                  March                      1108
                  December                   1072
                  July                       1043
                  February                    930
                  January                     438
2015              September                  1543
                  October                    1321
                  August                     1232
                  July                        939
                  December                    668
                  November                    301

In [145]:
# посмотрим, какое из средних значений, детей, вздослых и малышей,больше всего 

In [87]:
bookings[['adults', 'children', 'babies']].mean().idxmax()

'adults'

In [ ]:
# создадим колонку total_kids, в которой будет сумма children и babies

In [88]:
bookings['total_kids'] = bookings.children + bookings.babies

In [89]:
bookings.head()

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date,total_kids
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02,0.0
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02,0.0
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03,0.0


In [ ]:
# высчитаем наибольшее среднее знаачение total_kids для типа отелей и округлим его

In [90]:
bookings.groupby('hotel', as_index=False) \
        .agg({'total_kids': 'mean'}) \
        .round(2)

,hotel,total_kids
0,City Hotel,0.10
1,Resort Hotel,0.14


In [ ]:
# Не все бронирования завершились успешно (is_canceled), поэтому можно посчитать, сколько клиентов было потеряно в процессе. 
# Иными словами, посчитаем метрику Churn Rate.

In [95]:
bookings['has_kids'] = bookings.total_kids != 0

In [209]:
has_kids = bookings.query('has_kids == True').is_canceled.shape[0]
has_kids

9336

In [210]:
# посчитаем метрику Churn Rate сначала для гостей с детьми, потом без детей

In [211]:
has_kids_canceled = bookings.query('has_kids == True and is_canceled == 1').is_canceled.shape[0]
has_kids_canceled

3263

In [218]:
churn_rate_has_kids = has_kids_canceled / has_kids
round(churn_rate_has_kids*100, 2)   

34.95

In [213]:
no_kids = bookings.query('has_kids == False').is_canceled.shape[0]
no_kids

110054

In [214]:
no_kids_canceled = bookings.query('has_kids == False and is_canceled == 1').is_canceled.shape[0]
no_kids_canceled

40961

In [217]:
churn_rate_no_kids = no_kids_canceled / no_kids
round(churn_rate_no_kids*100, 2)   

37.22